In [ ]:
#default_exp expanding

# Expanding

> Expanding window operations

In [ ]:
#hide
import random

from nbdev.showdoc import *

In [ ]:
#export
from math import sqrt
from typing import Callable, Optional

import numpy as np
import pandas as pd
from numba import njit

from window_ops.rolling import _lt, _gt
from window_ops.utils import _expanding_std

In [ ]:
#export
@njit
def expanding_mean(x: np.ndarray) -> np.ndarray:
    n_samples = x.size
    out = np.empty(n_samples, dtype=np.float32)
    accum = 0.
    for i in range(n_samples):
        accum += x[i]
        out[i] = accum / (i+1)
    return out

In [ ]:
y = np.random.rand(100)
ys = pd.Series(y)

assert np.allclose(expanding_mean(y), ys.expanding().mean().values)

In [ ]:
#export
@njit
def expanding_std(x: np.ndarray) -> np.ndarray:
    out, _, _ = _expanding_std(x)
    return out

In [ ]:
y = np.random.rand(100)
ys = pd.Series(y)

assert np.allclose(expanding_std(y), ys.expanding().std().values, equal_nan=True)

In [ ]:
#exporti
@njit
def _expanding_comp(comp: Callable, x: np.ndarray) -> np.ndarray:
    n_samples = x.size
    out = np.empty(n_samples, dtype=np.float32)
    pivot = x[0]
    out[0] = pivot
    for i in range(1, n_samples):
        if comp(x[i], pivot):
            pivot = x[i]
        out[i] = pivot
    return out

In [ ]:
#export
def expanding_max(x: np.ndarray) -> np.ndarray:
    return _expanding_comp(_gt, x)

In [ ]:
y = np.random.rand(100)
ys = pd.Series(y)

assert np.allclose(expanding_max(y), ys.expanding().max().values)

In [ ]:
#export
def expanding_min(x: np.ndarray) -> np.ndarray:
    return _expanding_comp(_lt, x)

In [ ]:
y = np.random.rand(100)
ys = pd.Series(y)

assert np.allclose(expanding_min(y), ys.expanding().min().values)